In [2]:
import numpy as np
import scipy.io as sio
!pip install mne
import mne


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 11.0 MB/s eta 0:00:00
ERROR: Operation cancelled by user


ModuleNotFoundError: ignored

In [ ]:
mat_filename = 'ECoG_Handpose.mat'
raw_data = sio.loadmat(mat_filename)

In [ ]:
sampling_freq = 1200
ch_names = ["sample_time"] + [f'CH_{i}' for i in range(1, 61)] + ["paradigm_info"]
ch_types = ['misc'] + ['ecog'] * 60 + ['stim']
len(ch_names), len(ch_types)


(62, 62)

In [ ]:
info = mne.create_info(ch_names=ch_names, sfreq=1200, ch_types=ch_types)
raw = mne.io.RawArray(raw_data['y'][0:62, :], info)
raw.load_data().resample(sampling_freq)

Creating RawArray with float64 data, n_channels=62, n_times=507025
    Range : 0 ... 507024 =      0.000 ...   422.520 secs
Ready.
90 events found
Event IDs: [1 2 3]
90 events found
Event IDs: [1 2 3]


<RawArray | 62 x 507025 (422.5 s), ~239.9 MB, data loaded>

In [ ]:
event_labels = dict(fist_movement = 1, peace_movement = 2, open_hand = 3)
# find events in the MNE raw object
events = mne.find_events(raw, stim_channel="paradigm_info", shortest_event=1, verbose=True)
raw.load_data().resample(600)

90 events found
Event IDs: [1 2 3]
90 events found
Event IDs: [1 2 3]
90 events found
Event IDs: [1 2 3]


<RawArray | 62 x 253512 (422.5 s), ~120.0 MB, data loaded>

In [1]:
epochs = mne.Epochs(raw, events, tmin=-0.75, tmax=0.75, event_id=event_labels, preload=True, baseline=None)
epochs_train = epochs.copy().crop(tmin=0, tmax=sampling_freq)

NameError: ignored

In [ ]:
from mne.time_frequency import AverageTFR
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

In [ ]:
 # Classifier Pipeline CSP + LDA

clf = make_pipeline(CSP(n_components=4, reg=None, log=True, norm_trace=False),LinearDiscriminantAnalysis())
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

NameError: ignored

In [ ]:
tmin, tmax = -0.75, 0.75
n_cycles = 90
min_freq = 80
max_freq = 200
n_freqs = 10
freqs = np.linspace(min_freq, max_freq, n_freqs)
freq_ranges = list(zip(freqs[:-1], freqs[1:])) 

In [ ]:
window_spacing = (n_cycles / np.max(freqs) / 2.)
centered_w_times = np.arange(tmin, tmax, window_spacing)[1:]
n_windows = len(centered_w_times)
n_windows

6

In [ ]:
# Instantiate label encoder
le = LabelEncoder()

In [ ]:
freq_scores = np.zeros((n_freqs - 1,))

# Loop through each frequency range of interest
@#for freq, (fmin, fmax) in enumerate(freq_ranges):

    # Infer window size based on the frequency being used
    #  w_size = n_cycles / ((fmax + fmin) / 2.)  # in seconds

    # Apply band-pass filter to isolate the specified frequencies
    #raw_filter = raw.copy().filter(fmin, fmax, fir_design='firwin',
                                   skip_by_annotation='edge')

X = epochs.get_train_data()
   # y = le.fit_transform(epochs.events[:, 2])

    X = epochs.get_data()

    # Save mean scores over folds for each frequency and time window
    freq_scores[freq] = np.mean(cross_val_score(
        estimator=clf, X=X, y=y, scoring='accuracy', cv=cv), axis=0)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 80 - 93 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 80.00
- Lower transition bandwidth: 20.00 Hz (-6 dB cutoff frequency: 70.00 Hz)
- Upper passband edge: 93.33 Hz
- Upper transition bandwidth: 23.33 Hz (-6 dB cutoff frequency: 105.00 Hz)
- Filter length: 99 samples (0.165 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Computing rank from data with rank=None
    Using tolerance 5.2e+05 (2.2e-16 eps * 62 dim * 3.8e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    1.3s finished


Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.4e+05 (2.2e-16 eps * 62 dim * 3.9e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.4e+05 (2.2e-16 eps * 62 dim * 4e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.2e+05 (2.2e-16 eps * 62 dim * 3.8e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Us

...

Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.4e+05 (2.2e-16 eps * 62 dim * 3.9e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.4e+05 (2.2e-16 eps * 62 dim * 4e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.2e+05 (2.2e-16 eps * 62 dim * 3.8e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Us

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


Computing rank from data with rank=None
    Using tolerance 5.2e+05 (2.2e-16 eps * 62 dim * 3.8e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    3.5s finished


    Using tolerance 5.4e+05 (2.2e-16 eps * 62 dim * 3.9e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.4e+05 (2.2e-16 eps * 62 dim * 4e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.2e+05 (2.2e-16 eps * 62 dim * 3.8e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.4e+05 (2.2e-16 eps * 62 dim * 3.9e+19  max singular value)
    Estimated rank (mag): 62
    MAG: rank